llama 3.2   : Parameters size      : 3B, Input tokens : 128,000, Output tokens : ? <br>
openai - gpt-4o-mini : Parameters size(Apx) : 8B, Input tokens : 128,000, Output tokens : 16400 <br>
openai - text-embedding-ada-002 : Dimension : 1536

In [ ]:
pip install psycopg2-binary

In [ ]:
pip install psycopg2

In [43]:
from pyprojroot import here
import pandas as pd
import chromadb
from openai import OpenAI
from langchain.llms import Ollama
import psycopg2

In [15]:
# Database connection parameters
conn = psycopg2.connect(
    host="localhost",
    #database="postgres",
    database="vectordb",
    user="postgres",
    password="",
)
cursor = conn.cursor()

<b>Create LLAMA client</b>

In [44]:
ollama_endpoint = "http://localhost:11434"
#ollama_api_key = ""
llama_llm = Ollama(
    model="llama3.2",
    base_url=ollama_endpoint,
    #api_key=
)

/var/folders/79/bnb36vfn6wzdfrwd96krpm5m0000gn/T/ipykernel_76500/3168630985.py:3: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llama_llm = Ollama(


<b>Create GPT client</b>

In [46]:
gpt_llm = OpenAI(api_key="")

<b>Create vector DB client</b>

In [28]:
chroma_client = chromadb.PersistentClient(path=str(here("/Users/ashokarulsamy/Python Learning/itam/data/chroma_db/chroma_itam_ada")))

<b>Create collection in vector DB</b>

In [ ]:
#Already executed
collection = chroma_client.create_collection(name="itam_hw")

<b>Load data from csv file into data frame</b>

In [12]:
file_dir = here("/Users/ashokarulsamy/Python Learning/itam/data/itam_files/itam_hw.csv")
df = pd.read_csv(file_dir, nrows=100)

In [ ]:
filtered_df = df[df['Assigned_To'] == 'Ashok Arulsamy']

In [ ]:
filtered_df

<b>Create chunk and generate vector represenattion for chunk using embedding model</b>

In [16]:
#Already executed
docs = []
metadatas = []
ids = []
embeddings = []
for index, row in df.iterrows():
    output_str = ""
    # Treat each row as a separate chunk
    for col in df.columns:
        output_str += f"{col}: {row[col]},\n"
    response = gpt_llm.embeddings.create(input = [output_str], model="text-embedding-ada-002").data[0].embedding
    embeddings.append(response)
    docs.append(output_str)
    metadatas.append({"source": "itam_hw"})
    ids.append(f"id{index}")
    cursor.execute(
        "INSERT INTO itam1 (asset_detail, embedding) VALUES (%s, %s)",
        (output_str, response)
    )

conn.commit()
cursor.close()
conn.close()

In [ ]:
docs

In [ ]:
print(metadatas)
print(ids)

In [ ]:
embeddings[0][:10]

<b>Vector embedding - Load vector represenation in vector DB</b>

In [ ]:
#Already executed
collection.add(
    documents=docs,
    metadatas=metadatas,
    embeddings=embeddings,
    ids=ids
)

In [ ]:
print("Number of vectors in vectordb:", collection.count())

<b>Vector embedding for user query</b>

In [17]:
user_query = "I'm Ashok Arulsamy, list out assets assigned to me"
#user_query = "List of retired assets"
#user_query = "list of assets for which warrenty expires in the year 2025"
#user_query = "How many hardware assets are there" # try agent as semantic search will return huge volume of data
user_query_embeddings = gpt_llm.embeddings.create(input = [user_query], model="text-embedding-ada-002").data[0].embedding

In [ ]:
user_query_embeddings

<b>Create vector DB instance</b>

In [29]:
vectordb = chroma_client.get_collection(name="itam_hw")
vectordb.count()

100

<b>Semantic search for user query against vector DB</b>

In [30]:
retrieved_context = vectordb.query(
    query_embeddings = user_query_embeddings,
    n_results=5 #top_k
)

retrieved_context

{'ids': [['id23', 'id12', 'id1', 'id10', 'id6']],
 'embeddings': None,
 'documents': [['Asset_ID: HW024,\nAsset_Name: Card Device,\nAsset_Type: Hardware,\nManufacturer: Lenovo,\nModel: Model-0757,\nPurchase_Date: 2022-12-27,\nWarranty_Expiry: 2028-05-04,\nAssigned_To: Ashok Arulsamy,\nStatus: Retired,\n',
   'Asset_ID: HW013,\nAsset_Name: Majority Device,\nAsset_Type: Hardware,\nManufacturer: Lenovo,\nModel: Model-4924,\nPurchase_Date: 2022-04-22,\nWarranty_Expiry: 2029-10-02,\nAssigned_To: Ashok Arulsamy,\nStatus: In Use,\n',
   'Asset_ID: SW002,\nAsset_Name: Source Software,\nAsset_Type: Software,\nManufacturer: SAP,\nModel: Version-82.2,\nPurchase_Date: 2022-11-06,\nWarranty_Expiry: 2027-04-17,\nAssigned_To: Ashok Arulsamy,\nStatus: Active,\n',
   'Asset_ID: HW011,\nAsset_Name: Student Device,\nAsset_Type: Hardware,\nManufacturer: Lenovo,\nModel: Model-9315,\nPurchase_Date: 2024-09-04,\nWarranty_Expiry: 2026-03-25,\nAssigned_To: Anna Williams,\nStatus: Available,\n',
   'Asset_ID: H

In [37]:
def semantic_search(user_query_embeddings, top_k=5):
    try:
        conn = psycopg2.connect(
            host="localhost",
            #database="postgres",
            database="vectordb",
            user="postgres",
            password="password",
        )
        cursor = conn.cursor()

        # SQL query to perform semantic search
        # query = """
        # SELECT id, asset_detail, embedding, 
        #        1 - (embedding <-> %s) AS similarity -- Cosine similarity
        # FROM itam1
        # ORDER BY embedding <-> %s -- L2 (Euclidean) or cosine distance
        # LIMIT %s;
        # """

        query = """
        SELECT id, asset_detail, embedding, 
               1 - (embedding <-> %s::vector) AS similarity -- Cosine similarity
        FROM itam1
        ORDER BY embedding <-> %s::vector -- L2 (Euclidean) or cosine distance
        LIMIT %s;
        """
        
        # Execute the query
        cursor.execute(query, (user_query_embeddings, user_query_embeddings, top_k))
        results = cursor.fetchall()

        print(results)

        # Print the results
        for row in results:
            print(f"ID: {row[0]}, Content: {row[1]}, Similarity: {row[3]:.4f}")

        # formatted_results = [
        #     {"id": row[0], "content": row[1], "similarity": round(row[2], 4)}
        #     for row in results
        # ]

    except Exception as e:
        print("Error during semantic search:", e)
    finally:
        # Close the connection
        if conn:
            cursor.close()
            conn.close()


In [39]:
# 2. Perform similarity search using pgvector
def retrieve_similar_documents(query_embedding, top_k=5):
    try:
        conn = psycopg2.connect(
            host="localhost",
            #database="postgres",
            database="vectordb",
            user="postgres",
            password="password",
        )
        cursor = conn.cursor()

        # Convert query embedding to PostgreSQL array string
        query_embedding_pg = f"ARRAY[{', '.join(map(str, query_embedding))}]"

        # SQL query to perform similarity search
        query = f"""
        SELECT id, asset_detail, 1 - (embedding <-> {query_embedding_pg}::vector) AS similarity
        FROM itam1
        ORDER BY embedding <-> {query_embedding_pg}::vector
        LIMIT {top_k};
        """
        cursor.execute(query)
        results = cursor.fetchall()

        return [{"id": row[0], "content": row[1], "similarity": row[2]} for row in results]
    except Exception as e:
        print("Error during similarity search:", e)
        return []
    finally:
        cursor.close()
        conn.close()

In [41]:
#semantic_search(user_query_embeddings, top_k=5)
retrieved_context = retrieve_similar_documents(user_query_embeddings, top_k=5)
retrieved_context

[{'id': 24,
  'content': 'Asset_ID: HW024,\nAsset_Name: Card Device,\nAsset_Type: Hardware,\nManufacturer: Lenovo,\nModel: Model-0757,\nPurchase_Date: 2022-12-27,\nWarranty_Expiry: 2028-05-04,\nAssigned_To: Ashok Arulsamy,\nStatus: Retired,\n',
  'similarity': 0.37518651884399656},
 {'id': 13,
  'content': 'Asset_ID: HW013,\nAsset_Name: Majority Device,\nAsset_Type: Hardware,\nManufacturer: Lenovo,\nModel: Model-4924,\nPurchase_Date: 2022-04-22,\nWarranty_Expiry: 2029-10-02,\nAssigned_To: Ashok Arulsamy,\nStatus: In Use,\n',
  'similarity': 0.36930480825933343},
 {'id': 2,
  'content': 'Asset_ID: SW002,\nAsset_Name: Source Software,\nAsset_Type: Software,\nManufacturer: SAP,\nModel: Version-82.2,\nPurchase_Date: 2022-11-06,\nWarranty_Expiry: 2027-04-17,\nAssigned_To: Ashok Arulsamy,\nStatus: Active,\n',
  'similarity': 0.36772093473838385},
 {'id': 53,
  'content': 'Asset_ID: HW053,\nAsset_Name: Step Device,\nAsset_Type: Hardware,\nManufacturer: Asus,\nModel: Model-6226,\nPurchase_Date

<b>Generate user friendly message using LLAMA</b>

In [45]:
# Create a message with roles
messages = [
    {"role": "system", "content": "You are a knowledgeable assistant that provides detailed answers."},
    {"role": "user", "content": f"User's question: {user_query}"},
    {"role": "assistant", "content": f"Retrieved content: {retrieved_context}"}
]

# Convert messages to a single string
prompt = "\n".join([f"{msg['role'].capitalize()}: {msg['content']}" for msg in messages])

# Pass messages as input
response = llama_llm(prompt)
print(response)

/var/folders/79/bnb36vfn6wzdfrwd96krpm5m0000gn/T/ipykernel_76500/1309864738.py:12: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = llama_llm(prompt)


Based on the retrieved content, here are the assets assigned to you, Ashok Arulsamy:

1. Asset_ID: HW024
   - Asset_Name: Card Device
   - Asset_Type: Hardware
   - Manufacturer: Lenovo
   - Model: Model-0757
   - Purchase_Date: 2022-12-27
   - Warranty_Expiry: 2028-05-04
   - Status: Retired

2. Asset_ID: HW013
   - Asset_Name: Majority Device
   - Asset_Type: Hardware
   - Manufacturer: Lenovo
   - Model: Model-4924
   - Purchase_Date: 2022-04-22
   - Warranty_Expiry: 2029-10-02
   - Status: In Use

3. Asset_ID: SW002
   - Asset_Name: Source Software
   - Asset_Type: Software
   - Manufacturer: SAP
   - Model: Version-82.2
   - Purchase_Date: 2022-11-06
   - Warranty_Expiry: 2027-04-17
   - Status: Active

4. Asset_ID: HW053
   - Asset_Name: Step Device
   - Asset_Type: Hardware
   - Manufacturer: Asus
   - Model: Model-6226
   - Purchase_Date: 2022-10-14
   - Warranty_Expiry: 2029-05-23
   - Status: Available


<b>Generate user friendly message using GPT</b>

In [47]:
response = gpt_llm.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "You are a knowledgeable assistant that provides detailed answers."},
        {"role": "user", "content": f"User's question: {user_query}"},
        {"role": "assistant", "content": f"Retrieved content: {retrieved_context}"}
    ],
    temperature=0.0,  # Controls creativity (0.0 = deterministic, 1.0 = creative)
    max_tokens=1000,   # Adjust as needed
)

print("Generated Answer:")
print(response.choices[0].message.content)

Generated Answer:
Here is a list of assets assigned to you, Ashok Arulsamy:

1. **Asset ID:** HW024
   - **Asset Name:** Card Device
   - **Asset Type:** Hardware
   - **Manufacturer:** Lenovo
   - **Model:** Model-0757
   - **Purchase Date:** December 27, 2022
   - **Warranty Expiry:** May 4, 2028
   - **Status:** Retired

2. **Asset ID:** HW013
   - **Asset Name:** Majority Device
   - **Asset Type:** Hardware
   - **Manufacturer:** Lenovo
   - **Model:** Model-4924
   - **Purchase Date:** April 22, 2022
   - **Warranty Expiry:** October 2, 2029
   - **Status:** In Use

3. **Asset ID:** SW002
   - **Asset Name:** Source Software
   - **Asset Type:** Software
   - **Manufacturer:** SAP
   - **Model:** Version-82.2
   - **Purchase Date:** November 6, 2022
   - **Warranty Expiry:** April 17, 2027
   - **Status:** Active

4. **Asset ID:** HW053
   - **Asset Name:** Step Device
   - **Asset Type:** Hardware
   - **Manufacturer:** Asus
   - **Model:** Model-6226
   - **Purchase Date:** Oct